# Positivity of automorphic bundles

This note explores the possibility of proving that certain subvarieties of the good reduction of the Siegel variety are of log-general type. More precisely, let $X_g$ be a smooth toroidal compactification of the Siegel variety of genus $g \geq 1$ without level at $p$ over $\mathbb{F}_p$, we conjecture that any subvariety $V \subset X_g$ such that $\text{codim}(V) \leq g-1$ is of log-general type. To this end, we try to show that the exterior products $\Lambda^k \Omega^1_{X_g}$ of the cotangent bundle are $(\varphi,D)$-ample.

In [158]:
class SiegelVariety:
    def __init__(self,g,p):
        if p not in Primes():
            raise ValueError('The number ' + str(p) + ' is not prime')
        self.g = g
        self.p = p
        self.d = int(g*(g+1)/2)
        self.L = RootSystem("A"+str(self.g-1)).ambient_space() #Ag-1
        self.LG = RootSystem("C"+str(self.g)).ambient_space() #Cg
        self.W = WeylGroup(self.L)
        self.WG = WeylGroup(self.LG)

        A = WeylCharacterRing(['A',self.g-1])
        C = WeylCharacterRing(['C',self.g]) 
        self.phi_L = A.positive_roots()
        self.phi_G = C.positive_roots()
        self.Delta_L = A.simple_roots()
        self.Delta_G = C.simple_roots()
    
    #Change root convention
    def changeConvention(self,char):
        res = []
        for i in range(self.g-1,-1,-1):
            res.append(-char[i])
        return res
    
    #Test equality of characters
    def equality(self,el1,el2):
        for i in range(self.g):
            if el1[i] != el2[i]:
                return False
        return True
    
    #Test if a list contains an element
    def contains(self,lst,el):
        for x in lst:
            if self.equality(x,el):
                return True
        return False
    
    #Check L-dominance
    def Ldominant(self,char):
        for alpha in self.Delta_L:
            value = 0
            for j in range(self.g):
                value += char[j]*alpha[j]
            if value < 0:
                return False
        return True
    
    def extends_to_I(self, chi, I):
        chi = self.LG(chi).to_weight_space()
        simple = self.Delta_G
        SI = [simple[i] for i in I]
        extends = True
        for alpha in SI:
            alpha = alpha.associated_coroot()
            if chi.symmetric_form(alpha.to_weight_space()) != 0:
                extends = False
                break
        return extends
    
    #If it returns True, it means that the automorphic line bundle of weight char
    #is D-ample on the flag bundle of type I0
    def ample_flag_bundle(self, I0, char):
        char = self.changeConvention(char)
        I0 = [self.g-i for i in I0]
        simple = self.Delta_L
        Iprime = [simple[i] for i in I0]
        if len(I0) >0 and not self.extends_to_I(char,I0):
            raise ValueError('The character '+str(char)+' is not defined on the flag bundle of type '+ str(Iprime))
        for w in self.WG:
            for alpha in self.phi_G:
                alpha = alpha.associated_coroot()
                v = w.action(alpha)
                num = 0
                for j in range(self.g):
                    num += char[j]*v[j]
                den = 0
                for j in range(self.g):
                    den += char[j]*alpha[j]
                if den != 0:
                    if max(num/den,-num/den) > self.p-1:
                        return False
        for alpha in self.phi_G:
            if not self.contains(self.phi_L,alpha):
                alpha = alpha.associated_coroot()
                value = 0
                for j in range(self.g):
                    value += char[j]*alpha[j]
                if value >= 0:
                    return False
        for alpha in self.Delta_L:
            if alpha not in Iprime:
                value = 0
                for j in range(self.g):
                    value += char[j]*alpha[j]
                if value <= 0:
                    return False
        return True
    
    def ample_shimura(self, char, details = False):
        two_rho = sum(self.phi_L)
        mu = 2*self.L(char)+two_rho
        if details:
            print(mu)
        return self.Ldominant(mu) and self.ample_flag_bundle(I0 = [], char = mu)
    
    def positivity_of_cotangent_bundle(self,codimension):
        k = self.d-codimension
        plethysm = s([1]*k).plethysm(s[2])
        for char, n in list(plethysm):
            if len(char) <= self.g:
                char_norm = char + [0]*(self.g-len(char))
                res = self.ample_shimura(char_norm)
                print(str(char_norm)+' : '+str(res))
    
    def detailed_plethysm(self,partition):
        codimension = self.d-len(partition)
        plethysm = s(partition).plethysm(s[2])
        for char, n in list(plethysm):
            if len(char) <= self.g:
                char_norm = char + [0]*(self.g-len(char))
                print(str(char_norm)+' : ')
                res = self.ample_shimura(char_norm, details = True)
                print(res)

# Case $g = 2$

In [159]:
X = SiegelVariety(g = 2, p = 11)

In [160]:
X.positivity_of_cotangent_bundle(codimension = 0)

[3, 3] : True


In [161]:
X.positivity_of_cotangent_bundle(codimension = 1)

[3, 1] : True


In [162]:
X.positivity_of_cotangent_bundle(codimension = 2)

[2, 0] : False


In [163]:
X.ample_shimura((3,1), details = True)

(7, 1)


True

# Case $g = 3$

In [164]:
g = 3
d = int(g*(g+1)/2)
for codim in range(d):
    X = SiegelVariety(g = g, p = 17)
    print(str(codim)+' : ')
    X.positivity_of_cotangent_bundle(codim)

0 : 
[4, 4, 4] : True
1 : 
[4, 4, 2] : True
2 : 
[4, 3, 1] : False
3 : 
[4, 1, 1] : False
[3, 3, 0] : False
4 : 
[3, 1, 0] : False
5 : 
[2, 0, 0] : False


In the case where the codimension of $V$ is $2$, we do not get the $(\varphi,D)$-ampleness. Let's try to use a different Schur functor of the same length.

In [174]:
X.detailed_plethysm([2,2,2,2])

[8, 6, 2] : 
(18, 12, 2)
True
[7, 6, 3] : 
(16, 12, 4)
True
[8, 4, 4] : 
(18, 8, 6)
True
[6, 6, 4] : 
(14, 12, 6)
True


It works, with $S_{(2,2,2,2)}$ we only get $(\varphi,D)$-ample bundles

In [178]:
X.detailed_plethysm([2,2,2])

[7, 4, 1] : 
(16, 8, 0)
False
[8, 2, 2] : 
(18, 4, 2)
True
[6, 4, 2] : 
(14, 8, 2)
True
[4, 4, 4] : 
(10, 8, 6)
True
[6, 6, 0] : 
(14, 12, -2)
False


It doesn't seem to work in codimension $g = 3$

# Case $g = 4$

In [156]:
g = 4
d = int(g*(g+1)/2)
for codim in range(d):
    X = SiegelVariety(g = g, p = 17)
    print(str(codim)+' : ')
    X.positivity_of_cotangent_bundle(codim)

0 : 
[5, 5, 5, 5] : True
1 : 
[5, 5, 5, 3] : True
2 : 
[5, 5, 4, 2] : True
3 : 
[5, 4, 4, 1] : False
[5, 5, 2, 2] : True
4 : 
[5, 4, 2, 1] : False
[4, 4, 4, 0] : False
5 : 
[5, 3, 1, 1] : False
[4, 4, 2, 0] : False
6 : 
[5, 1, 1, 1] : False
[4, 3, 1, 0] : False
7 : 
[4, 1, 1, 0] : False
[3, 3, 0, 0] : False
8 : 
[3, 1, 0, 0] : False
9 : 
[2, 0, 0, 0] : False


# Case $g = 5$

In [157]:
g = 5
d = int(g*(g+1)/2)
for codim in range(d):
    X = SiegelVariety(g = g, p = 17)
    print(str(codim)+' : ')
    X.positivity_of_cotangent_bundle(codim)

0 : 
[6, 6, 6, 6, 6] : True
1 : 
[6, 6, 6, 6, 4] : True
2 : 
[6, 6, 6, 5, 3] : True
3 : 
[6, 6, 5, 5, 2] : False
[6, 6, 6, 3, 3] : True
4 : 
[6, 5, 5, 5, 1] : False
[6, 6, 5, 3, 2] : False
5 : 
[6, 5, 5, 3, 1] : False
[6, 6, 4, 2, 2] : False
[5, 5, 5, 5, 0] : False
6 : 
[6, 5, 4, 2, 1] : False
[6, 6, 2, 2, 2] : False
[5, 5, 5, 3, 0] : False
7 : 
[6, 4, 4, 1, 1] : False
[6, 5, 2, 2, 1] : False
[5, 5, 4, 2, 0] : False
8 : 
[6, 4, 2, 1, 1] : False
[5, 4, 4, 1, 0] : False
[5, 5, 2, 2, 0] : False
9 : 
[6, 3, 1, 1, 1] : False
[5, 4, 2, 1, 0] : False
[4, 4, 4, 0, 0] : False
10 : 
[6, 1, 1, 1, 1] : False
[5, 3, 1, 1, 0] : False
[4, 4, 2, 0, 0] : False
11 : 
[5, 1, 1, 1, 0] : False
[4, 3, 1, 0, 0] : False
12 : 
[4, 1, 1, 0, 0] : False
[3, 3, 0, 0, 0] : False
13 : 
[3, 1, 0, 0, 0] : False
14 : 
[2, 0, 0, 0, 0] : False
